In [1]:
from influxdb import InfluxDBClient

In [2]:
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'RCL')

In [3]:
chosen_time='45d'

In [4]:
from enum import Enum
class Engine():
    
    def __init__(self, speed):
        self.speed = speed
    def stop(self):
        self.speed = 0
        return "Stopped"
    def increase_speed_by(self, n):
        self.speed+=n
        return "Increased by"+str(n)
    def decrease_speed_by(self, n):
        self.speed-=n
        return "Decreased by"+str(n)
    
def CheckForStop(engine,time_period, minDWeight,maxDWeight,minNDWeight,maxNDWeight,minSpeed,maxSpeed):
    """Stops engine if weights are enormal (less than of an empty container or more than MaxWeight)
    
    Keyword arguments:
    engine -- chosen engine for modifying speed
    time_period -- time_period for searching enormals
    minDWeight -- weight of an empty container of defect bearings
    maxDWeight -- weight of a full container of defect bearings
    minNDWeight -- weight of an empty container of non-defect bearings
    maxNDWeight -- weight of a full container of non-defect bearings
    minSpeed -- minimal possible speed
    maxSpeed -- maximum possible speed
    
    """
    result=client.query('select max(weightD1) as wd1, max(weightD2) as wd2, max(weightND1) as wnd1, max(speedE1) as se1'+
                        ' from RCL where time>now()-'+time_period).get_points()
    for val in result:
        if (val['wd1']>maxDWeight or val['wd2']>maxDWeight or val['wnd1']>maxNDWeight or val['se1']>maxSpeed):
            print('Max')
            engine.stop()
    
    result1=client.query('select min(weightD1) as wd1, min(weightD2) as wd2, min(weightND1) as wnd1, min(speedE1) as se1'+
                        ' from RCL where time>now()-'+time_period).get_points()
    for val in result1:
        if (float(val['wd1'])<minDWeight or float(val['wd2'])<minDWeight or
            float(val['wnd1'])<minNDWeight or float(val['se1'])<minSpeed):
            print('Min')
            engine.stop() 

In [5]:
eng = Engine(1500)

In [6]:
CheckForStop(eng, '1d', 140,400,300,1200,20,10000)

ConnectionError: HTTPConnectionPool(host='localhost', port=8086): Max retries exceeded with url: /query?q=select+max%28weightD1%29+as+wd1%2C+max%28weightD2%29+as+wd2%2C+max%28weightND1%29+as+wnd1%2C+max%28speedE1%29+as+se1+from+RCL+where+time%3Enow%28%29-1d&db=RCL (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000262D8CD9DC0>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))

In [ ]:
eng.speed

In [ ]:
maxDWeight = 400
maxNDWeight = 1200
def CountOfProduction(constwd1, constwd2, constwnd1, weightofcorrect, weightofdefect, timespan=None):
    """Calculates count of bearings in containers
    
    Keyword arguments:
    constwd1 -- weight of first and empty container of defect bearings
    constwd2 -- weight of second and empty container of defect bearings
    constwnd1 -- weight of an empty container of non-defect bearings
    weightofcorrect -- weight of 1 full bearing
    weightofdefect -- weight of 1 not full bearing
    timespan -- period of time (default = None)
    
    """
    result = client.query('select last(weightD1) as wd1, last(weightD2) as wd2,'+
                              ' last(weightND1) as wnd1 from RCL').get_points()
    for val in result:
        wd1 = val['wd1']
        wd2 = val['wd2']
        wnd1 = val['wnd1']
    wd1-=constwd1
    wd2-=constwd2
    wnd1-=constwnd1
    if timespan == None:
        return int(wd1/weightofdefect), int(wd2/weightofcorrect), int(wnd1/weightofcorrect)
    else:
        result = client.query('select first(weightD1) as wd1, first(weightD2) as wd2,'+
                              ' first(weightND1) as wnd1 from RCL where time>now()-'+timespan).get_points()
        for val in result:
            wd1_before = val['wd1']
            wd2_before = val['wd2']
            wnd1_before = val['wnd1']
        wd1_before-=constwd1
        wd2_before-=constwd2
        wnd1_before-=constwnd1
        if wd1-wd1_before<0:
            wd1 = maxDWeight-wd1_before+wd1-constwd1
        else:
            wd1 = wd1-wd1_before
        if wd2-wd2_before<0:
            wd2 = maxDWeight - wd2_before+wd2-constwd2
        else:
            wd2 = wd2-wd2_before
        if wnd1 - wnd1_before<0:
            wnd1 = maxNDWeight - wnd1_before+wnd1-constwnd1
        else:
            wnd1-=wnd1_before
        return int(wd1/weightofdefect), int(wd2/weightofcorrect), int(wnd1/weightofcorrect)

In [ ]:
defect1, defect2, nondefect = CountOfProduction(124, 124, 311, 0.1, 0.094, '1000m')

In [ ]:
def PercentageOfNonCorrect(d1, d2, nd):
    return (d1+d2)/(d1+d2+nd)*100

In [ ]:
PercentageOfNonCorrect(defect1, defect2, nondefect)

In [ ]:
def MakeDecisionAboutSpeed(engine,delta_speed,maxP,minP,
                           constwd1, constwd2, constwnd1, weightofcorrect, weightofdefect, time_span=None):
    """Makes a Decision of speed depending on percentage of defects
    
    Keyword arguments:
    engine -- chosen engine for modifying speed
    delta_speed -- change (a step) in speed which is done if percentage is too low or too high
    maxP -- max Percentage of defects
    minP -- min Percentage of defects
    constwd1 -- weight of first and empty container of defect bearings
    constwd2 -- weight of second and empty container of defect bearings
    constwnd1 -- weight of an empty container of non-defect bearings
    weightofcorrect -- weight of 1 full bearing
    weightofdefect -- weight of 1 not full bearing
    timespan -- period of time (default = None)
    
    """
    d1, d2, nd = CountOfProduction(constwd1, constwd2, constwnd1, weightofcorrect, weightofdefect, time_span)
    print("First Defect Container Count: "+str(d1))
    print("Second Defect Container Count: "+str(d2))
    print("Non-Defect Container Count: "+str(nd))
    percentile = PercentageOfNonCorrect(d1, d2, nd)
    print("Percentage: ", percentile)
    if percentile < minP:
        engine.increase_speed_by(delta_speed)
        print('Increased! Current speed = '+ str(engine.speed))
    elif percentile > maxP:
        engine.decrease_speed_by(delta_speed)
        print('Decreased! Current speed = '+ str(engine.speed))

In [ ]:
MakeDecisionAboutSpeed(eng, 100, 9, 3, 124, 124, 300, 0.1, 0.094, '120m')

In [ ]:
eng.speed